# Лабораторная работа №2

Выполнила Вероника Царева, группа БКЛ223.

При работе использовался python версии 3.8.4

### Задание

Необходимо написать программу, которая генерирует XML-представление дерева зависимостей предложения по его записи в формате CoNLL-U.

**Комментарии к реализации**

Работа кода проиллюстрирована на предложении «*Я очень люблю пить холодный кофе со льдом и карамельным сиропом по утрам, пока иду в университет к первой паре.*».

Запись данного предложения в формате CoNLL-U была получена с помощью [онлайн-парсинга](https://lindat.mff.cuni.cz/services/udpipe/run.php) предложений в UDPipe для русского языка (доступен только с vpn, модель russian-syntagrus-ud-2.12-230717) и находится в файле ```example_sentence.conllu```.

В xml-представлении дерева зависимостей будут отражены: токен слова (token), его часть речи (pos), название грамматического отношения слова к своей вершине (deprel).

Если у слова есть зависимые, то он будет заключен в тэг ``` <head> </head> ```. Перед вершиной группы на том же уровне (отступе и строке) будет открываться тэг ``` <phrase> ```, а заканчиваться он будет после последнего зависимого вершины. Это нужно, чтобы можно было в дальнейшем удобно работать с этой структурой (см. Раздел «Примеры использования»).

Если у слова нет зависимых, то он будет заключен в тэг ```<dependent> </dependent>```.

Если вершина нулевая, то она будет заключена в тэг ```<root> </root>```.

Отступ каждого уровня от предыдущего равен 4м пробелам.

### Реализация

Импорт и установка необходимых модулей.

In [1]:
%pip install treelib

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import typing
from typing import Union
from treelib import Tree


Создание названий для использующихся типов в функциях.

In [3]:
Nodes = typing.Dict[
    int, typing.List[Union[typing.Tuple[str, str, str], typing.List[int]]]
]


Функция для обработки предложения в формате ConLL-U.

In [4]:
def make_dependency_tree(filename: str) -> Nodes:
    # разделяем файл по строкам, где строки -- информация о каждом слове
    # в предложении
    words = []
    with open(filename) as file:
        words = file.read().split("\n")

    # делаем сплит каждой строки по табу, чтобы получить отдельные
    # характеристики слова
    word_data = [word.split("\t") for word in words]

    # заводим словарь, где ключ -- номер токена, а значение это список из
    # кортежа (токен, часть речи и грамматическое отношения) и
    # списка (номеров детей)

    # добавляем "нулевую" вершину, которая является корнем дерева
    nodes = {}
    nodes[0] = [("-", "root", "-"), []]

    # заполняем словарь номерами токенов и кортежами с информацией о токене
    for i in range(len(word_data)):
        token, pos, deprel = word_data[i][1], word_data[i][3], word_data[i][7]
        nodes[i + 1] = [(token, pos, deprel), []]

    for i in range(len(word_data)):
        # индекс родителя
        parent_idx = int(word_data[i][6])
        # добавляем номера детей в список своим родителям
        nodes[parent_idx][1].append(i + 1)

    return nodes


Функция для генерации xml-представления дерева.

In [5]:
def xml_generator(filename: str, nodes: Nodes, idx: int, shift=0) -> None:
    # токен
    token = nodes[idx][0][0]

    # часть речи
    pos = nodes[idx][0][1]

    # грамматическое отношение
    deprel = nodes[idx][0][2]

    # список детей
    children = nodes[idx][1]

    # если у токена есть зависимые, то он будет записываться в файл с тэгом
    # <head> </head>
    if len(children) != 0:
        # для корневой вершины тэг будет <root> </root>
        if pos == "root":
            # открываем тэг
            filename.write(f'{" "*shift}<root>\n')

            # делаем рекурсивный обход по всем детям
            for child in children:
                xml_generator(filename, nodes, child, shift + 4)

            # закрываем тэг
            filename.write(f'{" "*shift}</root>\n')
        else:
            # открываем тэг c отступом shift и записываем атрибуты токена
            # (часть речи и грамматическое отношение) и сам токен
            filename.write(f'{" "*shift}<phrase> ')
            filename.write(f'<head pos="{pos}" deprel="{deprel}">{token}</head>\n')

            # делаем рекурсивный обход по всем детям
            for child in children:
                xml_generator(filename, nodes, child, shift + 4)

            # закрываем тэг
            filename.write(f'{" "*shift}</phrase>\n')
    # без зависимых у токена будет тэг <dependent> </dependent>
    else:
        # открываем тэг c отступом shift и записываем атрибуты токена
        # (часть речи и грамматическое отношение), сам токен и закрываем тэг
        filename.write(
            f'{" "*shift}<dependent pos="{pos}" deprel="{deprel}">{token}</dependent>\n'
        )


In [6]:
# парсинг предложения в формате conllu
dependency_tree = make_dependency_tree('example_sentence.conllu')


In [7]:
# запись предложения в формале xml в файл
with open('dependency_tree.xml', 'w') as file:
    file.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    xml_generator(file, dependency_tree, 0)


In [8]:
# вывод дерева в формате xml
with open('dependency_tree.xml', 'r') as file:
    lines = file.readlines()
    print(*lines)


<?xml version="1.0" encoding="UTF-8"?>
 <root>
     <phrase> <head pos="VERB" deprel="root">люблю</head>
         <dependent pos="PRON" deprel="nsubj">Я</dependent>
         <dependent pos="ADV" deprel="advmod">очень</dependent>
         <phrase> <head pos="VERB" deprel="xcomp">пить</head>
             <phrase> <head pos="NOUN" deprel="obj">кофе</head>
                 <dependent pos="ADJ" deprel="amod">холодный</dependent>
                 <phrase> <head pos="NOUN" deprel="nmod">льдом</head>
                     <dependent pos="ADP" deprel="case">со</dependent>
                     <phrase> <head pos="NOUN" deprel="conj">сиропом</head>
                         <dependent pos="CCONJ" deprel="cc">и</dependent>
                         <dependent pos="ADJ" deprel="amod">карамельным</dependent>
                     </phrase>
                 </phrase>
             </phrase>
             <phrase> <head pos="NOUN" deprel="obl">утрам</head>
                 <dependent pos="ADP" deprel="case"

### Визуализация результата

Функция для визуализации результатов.

In [9]:
def print_tree(tree: Tree, dependency_tree: Nodes, parent = 0) -> None:
    # рекурсивно перебираем детей текущего родителя
    for child in dependency_tree[parent][1]:
        # создаем ярлык для листа
        leaf = f"{dependency_tree[child][0][0]}: {dependency_tree[child][0][1]}, {dependency_tree[child][0][2]}"
        
        # добавляем вершину в дерево
        tree.create_node(leaf, child, parent=parent)
        
        # запускаем рекурсию для детей текущего ребенка
        print_tree(tree, dependency_tree, child)


In [10]:
# создание объекта Tree
tree = Tree()

# добавляем корень
tree.create_node("root", 0)

# вызов функции для добавления вершин в дерево
print_tree(tree, dependency_tree, 0)

# визуализация дерева
print(tree)


root
└── люблю: VERB, root
    ├── .: PUNCT, punct
    ├── Я: PRON, nsubj
    ├── иду: VERB, advcl
    │   ├── ,: PUNCT, punct
    │   ├── паре: NOUN, obl
    │   │   ├── к: ADP, case
    │   │   └── первой: ADJ, amod
    │   ├── пока: SCONJ, mark
    │   └── университет: NOUN, obl
    │       └── в: ADP, case
    ├── очень: ADV, advmod
    └── пить: VERB, xcomp
        ├── кофе: NOUN, obj
        │   ├── льдом: NOUN, nmod
        │   │   ├── сиропом: NOUN, conj
        │   │   │   ├── и: CCONJ, cc
        │   │   │   └── карамельным: ADJ, amod
        │   │   └── со: ADP, case
        │   └── холодный: ADJ, amod
        └── утрам: NOUN, obl
            └── по: ADP, case



### Примеры использования дерева в формате xml

Установка и импорт необходимых модулей.

In [11]:
from bs4 import BeautifulSoup


С помощью такого представления дерева мы можем быстро находить все вершины групп в дереве.

In [12]:
# открытие файла на чтение
with open('dependency_tree.xml', 'r') as file:
    tree = file.read() 

# делаем суп из файла
soup = BeautifulSoup(tree, 'xml')


In [13]:
# ищем все вершины
heads = soup.find_all("head")

for word in heads:
    print(word.get_text())


люблю
пить
кофе
льдом
сиропом
утрам
иду
университет
паре


In [14]:
# ищем все вершины-существительные

noun_heads = soup.find_all("head", pos="NOUN")

for word in noun_heads:
    print(word.get_text())


кофе
льдом
сиропом
утрам
университет
паре


In [15]:
# ищем все зависимые-прилагательные

adj_dependet = soup.find_all("dependent", pos="ADJ")

for word in adj_dependet:
    print(word.get_text())


холодный
карамельным
первой
